In [1]:
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

client = openai.Client()

In [2]:
vector_store = client.beta.vector_stores.create(name = 'Apostilas Asimov')

In [3]:
files = [
    'arquivos/Explorando a API da OpenAI.pdf',
    'arquivos/Explorando o Universo das IAs com Hugging Face.pdf'
]

file_stream = [open(f, 'rb') for f in files]

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id,
    files=file_stream
)

In [4]:
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=2, failed=0, in_progress=0, total=2)


In [11]:
assistant = client.beta.assistants.create(
    name='Tutor',
    instructions='''
        Você é um tutor de uma escola de programação.
        Você é ótimo para responder perguntas teóricas sobre a api da OpenAI e sobre a utilização
        da biblioteca Hugging Face com Python. Você utiliza as apostilas dos cursos para basear suas respostas.
        Caso você não encontre as respostas nas apostilas informadas, você fala que não sabe a resposta. 
    ''',
    tools=[{'type':'file_search'}],
    tool_resources={'file_search':{'vector_store_ids':[vector_store.id]}},
    model='gpt-3.5-turbo-0125'
)

In [12]:
thread = client.beta.threads.create()

In [38]:
mensagem_texto = 'Segundo o documento fornecido, o que é o Hugging Face?'
mensagem_texto = 'Segundo o documento fornecido, como utilizar assistants com Python?'

messages = client.beta.threads.messages.create(
    thread_id=thread.id,
    role='user',
    content=mensagem_texto
)

In [39]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions='O nome do usuário é Anna e ela é um usuário Premium'
)

In [40]:
import time

while run.status in ['queued', 'in_progress', 'cancelling']:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
print(run.status)

completed


In [41]:
if run.status == 'completed':
    mensagens = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(mensagens)
else:
    print('Erro', run.status)

SyncCursorPage[Message](data=[Message(id='msg_c8LznfC7dfx4MQsVdubGN2rD', assistant_id='asst_NaQOlPDwR6WIWfniFhtXDODg', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=864, file_citation=FileCitation(file_id='file-PovZdD8NM7TgpkKNvVe7sr'), start_index=852, text='【8:0†source】', type='file_citation'), FileCitationAnnotation(end_index=876, file_citation=FileCitation(file_id='file-PovZdD8NM7TgpkKNvVe7sr'), start_index=864, text='【8:1†source】', type='file_citation'), FileCitationAnnotation(end_index=888, file_citation=FileCitation(file_id='file-PovZdD8NM7TgpkKNvVe7sr'), start_index=876, text='【8:3†source】', type='file_citation')], value='Para utilizar assistentes com Python, é possível seguir os seguintes passos conforme o documento "Explorando a API da OpenAI.pdf":\n\n1. Inicialize o cliente da OpenAI utilizando o código fornecido no documento.\n2. Crie um assistente com uma instrução específica, como por exemplo, um assis

In [42]:
print(mensagens.data[0].content[0].text.value)

Para utilizar assistentes com Python, é possível seguir os seguintes passos conforme o documento "Explorando a API da OpenAI.pdf":

1. Inicialize o cliente da OpenAI utilizando o código fornecido no documento.
2. Crie um assistente com uma instrução específica, como por exemplo, um assistente que responda perguntas de matemática e tenha acesso à ferramenta de interpretação de código.
3. Crie uma thread de comunicação com o assistente, adicionando mensagens a ela.
4. Solicite ao assistente para rodar a thread e aguarde a finalização do processo.
5. Verifique a resposta gerada pelo assistente, solicitando a lista de mensagens da thread e extraindo a mensagem final em formato de texto.

Esses passos envolvem a criação do assistente, a interação por meio de threads e a obtenção das respostas geradas pelo modelo de assistente da OpenAI em Python【8:0†source】【8:1†source】【8:3†source】.


In [43]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

In [44]:
for step in run_steps.data:
    print('=== Step:', step.step_details.type)
    if step.step_details.type == 'tool_calls':
        for tool_call in step.step_details.tool_calls:
            if tool_call.type == 'file_search':
                print(tool_call)
            else:
                print('----------')
                print(tool_call.code_interpreter.input)
                print('----------')
                print('Result')
                print(tool_call.code_interpreter.outputs[0].logs)
    if step.step_details.type == 'message_creation':
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id
        )
        print(message.content[0].text.value)
            

=== Step: message_creation
Para utilizar assistentes com Python, é possível seguir os seguintes passos conforme o documento "Explorando a API da OpenAI.pdf":

1. Inicialize o cliente da OpenAI utilizando o código fornecido no documento.
2. Crie um assistente com uma instrução específica, como por exemplo, um assistente que responda perguntas de matemática e tenha acesso à ferramenta de interpretação de código.
3. Crie uma thread de comunicação com o assistente, adicionando mensagens a ela.
4. Solicite ao assistente para rodar a thread e aguarde a finalização do processo.
5. Verifique a resposta gerada pelo assistente, solicitando a lista de mensagens da thread e extraindo a mensagem final em formato de texto.

Esses passos envolvem a criação do assistente, a interação por meio de threads e a obtenção das respostas geradas pelo modelo de assistente da OpenAI em Python【8:0†source】【8:1†source】【8:3†source】.
=== Step: tool_calls
FileSearchToolCall(id='call_6SaYS9lgly13J1tIdxryYPxw', file_se

In [45]:
mensagens = client.beta.threads.messages.list(
    thread_id=thread.id
)

mensagens = list(mensagens)[0].content[0].text
print(mensagens)

anotacoes = mensagens.annotations
print(anotacoes)

citacoes = []
for index, anotacao in enumerate(anotacoes):
    mensagens.value = mensagens.value.replace(anotacao.text, f'[{index}]')
    if file_cit := getattr(anotacao, 'file_citation', None):
        file = client.files.retrieve(file_cit.file_id)
        citacoes.append(f'[{index}] {file.filename}')
        print(file_cit)
citacoes = '\n'.join(citacoes)
mensagens.value = f'{mensagens.value}\n{citacoes}'

Text(annotations=[FileCitationAnnotation(end_index=864, file_citation=FileCitation(file_id='file-PovZdD8NM7TgpkKNvVe7sr'), start_index=852, text='【8:0†source】', type='file_citation'), FileCitationAnnotation(end_index=876, file_citation=FileCitation(file_id='file-PovZdD8NM7TgpkKNvVe7sr'), start_index=864, text='【8:1†source】', type='file_citation'), FileCitationAnnotation(end_index=888, file_citation=FileCitation(file_id='file-PovZdD8NM7TgpkKNvVe7sr'), start_index=876, text='【8:3†source】', type='file_citation')], value='Para utilizar assistentes com Python, é possível seguir os seguintes passos conforme o documento "Explorando a API da OpenAI.pdf":\n\n1. Inicialize o cliente da OpenAI utilizando o código fornecido no documento.\n2. Crie um assistente com uma instrução específica, como por exemplo, um assistente que responda perguntas de matemática e tenha acesso à ferramenta de interpretação de código.\n3. Crie uma thread de comunicação com o assistente, adicionando mensagens a ela.\n4. 

In [46]:
print(mensagens.value)

Para utilizar assistentes com Python, é possível seguir os seguintes passos conforme o documento "Explorando a API da OpenAI.pdf":

1. Inicialize o cliente da OpenAI utilizando o código fornecido no documento.
2. Crie um assistente com uma instrução específica, como por exemplo, um assistente que responda perguntas de matemática e tenha acesso à ferramenta de interpretação de código.
3. Crie uma thread de comunicação com o assistente, adicionando mensagens a ela.
4. Solicite ao assistente para rodar a thread e aguarde a finalização do processo.
5. Verifique a resposta gerada pelo assistente, solicitando a lista de mensagens da thread e extraindo a mensagem final em formato de texto.

Esses passos envolvem a criação do assistente, a interação por meio de threads e a obtenção das respostas geradas pelo modelo de assistente da OpenAI em Python[0][1][2].
[0] Explorando a API da OpenAI.pdf
[1] Explorando a API da OpenAI.pdf
[2] Explorando a API da OpenAI.pdf


In [47]:
citacoes

'[0] Explorando a API da OpenAI.pdf\n[1] Explorando a API da OpenAI.pdf\n[2] Explorando a API da OpenAI.pdf'

In [48]:
mensagens.value

'Para utilizar assistentes com Python, é possível seguir os seguintes passos conforme o documento "Explorando a API da OpenAI.pdf":\n\n1. Inicialize o cliente da OpenAI utilizando o código fornecido no documento.\n2. Crie um assistente com uma instrução específica, como por exemplo, um assistente que responda perguntas de matemática e tenha acesso à ferramenta de interpretação de código.\n3. Crie uma thread de comunicação com o assistente, adicionando mensagens a ela.\n4. Solicite ao assistente para rodar a thread e aguarde a finalização do processo.\n5. Verifique a resposta gerada pelo assistente, solicitando a lista de mensagens da thread e extraindo a mensagem final em formato de texto.\n\nEsses passos envolvem a criação do assistente, a interação por meio de threads e a obtenção das respostas geradas pelo modelo de assistente da OpenAI em Python[0][1][2].\n[0] Explorando a API da OpenAI.pdf\n[1] Explorando a API da OpenAI.pdf\n[2] Explorando a API da OpenAI.pdf'